In [4]:
import pandas as pd
from datasets import load_dataset, Audio


df = pd.read_csv("../data/cv-corpus-21.0-delta-2025-03-14/en/validated.tsv", delimiter="\t")
commonvoice = load_dataset("audiofolder", data_dir=f"../data/cv-corpus-21.0-delta-2025-03-14/validated_clips")
commonvoice = commonvoice.cast_column("audio", Audio(sampling_rate=16000)) 



In [5]:
from transformers import AutoProcessor, Wav2Vec2ForCTC
import torch
import librosa
import soundfile
import pickle
from tqdm import tqdm
import numpy as np
model_checkpoint = "facebook/wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_checkpoint)
processor = AutoProcessor.from_pretrained(model_checkpoint)

clip_path = "../data/cv-corpus-21.0-delta-2025-03-14/en"
output_dict = {}
for cv in tqdm(commonvoice["train"]):
    # audio file is decoded on the fly
    inputs = processor(cv["audio"]["array"], sampling_rate=cv["audio"]["sampling_rate"], return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    last_hidden_states = outputs.hidden_states[-1] # i think these are the embeddings that I am going to want
    output_dict[cv['audio']['path'].split("/")[-1].split(".")[0]] = last_hidden_states

with open("commonvoice_embeddings.pkl", 'wb') as file:
    # Use pickle.dump() to serialize and save the dictionary
    pickle.dump(output_dict, file)
    file.close()


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 249/249 [02:15<00:00,  1.84it/s]


In [6]:
# import torch
# import torchaudio

# bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
# model = bundle.get_model()
# for cv in tqdm(commonvoice["train"]):

#     waveform, sample_rate = torchaudio.load(SPEECH_FILE)
#     if sample_rate != bundle.sample_rate:
#         waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
#     with torch.inference_mode():
#         features, _ = model.extract_features(waveform)
